### Import thư viện

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Đọc dữ liệu từ file csv

In [9]:
df = pd.read_csv("./Sample - Superstore.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 2944: invalid start byte

## Tiền xử lý

**In 5 dòng đầu tiên của dữ liệu**

In [43]:
df.head()

,id,title,url,is_paid,num_subscribers,avg_rating,avg_rating_recent,rating,num_reviews,is_wishlisted,num_published_lectures,num_published_practice_tests,created,published_time,discount_price__amount,discount_price__currency,discount_price__price_string,price_detail__amount,price_detail__currency,price_detail__price_string
0,762616,The Complete SQL Bootcamp 2020: Go from Zero t...,/course/the-complete-sql-bootcamp/,True,295509,4.66019,4.67874,4.67874,78006,False,84,0,2016-02-14T22:57:48Z,2016-04-06T05:16:11Z,455.0,INR,₹455,8640.0,INR,"₹8,640"
1,937678,Tableau 2020 A-Z: Hands-On Tableau Training fo...,/course/tableau10/,True,209070,4.58956,4.60015,4.60015,54581,False,78,0,2016-08-22T12:10:18Z,2016-08-23T16:59:49Z,455.0,INR,₹455,8640.0,INR,"₹8,640"
2,1361790,PMP Exam Prep Seminar - PMBOK Guide 6,/course/pmp-pmbok6-35-pdus/,True,155282,4.59491,4.59326,4.59326,52653,False,292,2,2017-09-26T16:32:48Z,2017-11-14T23:58:14Z,455.0,INR,₹455,8640.0,INR,"₹8,640"
3,648826,The Complete Financial Analyst Course 2020,/course/the-complete-financial-analyst-course/,True,245860,4.54407,4.53772,4.53772,46447,False,338,0,2015-10-23T13:34:35Z,2016-01-21T01:38:48Z,455.0,INR,₹455,8640.0,INR,"₹8,640"
4,637930,An Entire MBA in 1 Course:Award Winning Busine...,/course/an-entire-mba-in-1-courseaward-winning...,True,374836,4.47080,4.47173,4.47173,41630,False,83,0,2015-10-12T06:39:46Z,2016-01-11T21:39:33Z,455.0,INR,₹455,8640.0,INR,"₹8,640"


**Thông tin số dòng và số cột**

In [44]:
df.shape

(22853, 20)

**Mô tả dữ liệu**

In [45]:
df.describe()

,id,num_subscribers,avg_rating,avg_rating_recent,rating,num_reviews,num_published_lectures,num_published_practice_tests,discount_price__amount,price_detail__amount
count,2.285300e+04,22853.000000,22853.000000,22853.000000,22853.000000,22853.000000,22853.00000,22853.000000,21024.000000,22356.000000
mean,1.818466e+06,3205.448256,3.952356,3.937739,3.937739,270.277557,34.91721,0.375224,486.266077,4445.517982
std,9.273525e+05,11051.296472,0.875152,0.888605,0.888605,2048.788093,48.65282,1.160939,234.100393,3098.531678
min,2.762000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,455.000000,1280.000000
25%,1.090694e+06,76.000000,3.750000,3.732460,3.732460,8.000000,11.00000,0.000000,455.000000,1280.000000
50%,1.824268e+06,559.000000,4.150000,4.148680,4.148680,27.000000,22.00000,0.000000,455.000000,3200.000000
75%,2.604580e+06,2483.000000,4.435480,4.433520,4.433520,98.000000,41.00000,0.000000,455.000000,8640.000000
max,3.486006e+06,564444.000000,5.000000,5.000000,5.000000,188941.000000,699.00000,6.000000,3200.000000,12800.000000


**Thông tin cột**

In [46]:
df.columns

Index(['id', 'title', 'url', 'is_paid', 'num_subscribers', 'avg_rating',
       'avg_rating_recent', 'rating', 'num_reviews', 'is_wishlisted',
       'num_published_lectures', 'num_published_practice_tests', 'created',
       'published_time', 'discount_price__amount', 'discount_price__currency',
       'discount_price__price_string', 'price_detail__amount',
       'price_detail__currency', 'price_detail__price_string'],
      dtype='object')

**Kiểu dữ liệu của từng cột**

In [47]:
df.dtypes

id                                int64
title                            object
url                              object
is_paid                            bool
num_subscribers                   int64
avg_rating                      float64
avg_rating_recent               float64
rating                          float64
num_reviews                       int64
is_wishlisted                      bool
num_published_lectures            int64
num_published_practice_tests      int64
created                          object
published_time                   object
discount_price__amount          float64
discount_price__currency         object
discount_price__price_string     object
price_detail__amount            float64
price_detail__currency           object
price_detail__price_string       object
dtype: object

**Số giá trị null của từng cột**

In [48]:
df.isnull().sum()

id                                 0
title                              0
url                                0
is_paid                            0
num_subscribers                    0
avg_rating                         0
avg_rating_recent                  0
rating                             0
num_reviews                        0
is_wishlisted                      0
num_published_lectures             0
num_published_practice_tests       0
created                            0
published_time                     0
discount_price__amount          1829
discount_price__currency        1829
discount_price__price_string    1829
price_detail__amount             497
price_detail__currency           497
price_detail__price_string       497
dtype: int64

**Loại bỏ các dòng có xuất hiện it nhất 1 giá trị null trong dataframe**

Do dữ liệu có hơn 20000 dòng nên việc loại bỏ các giá trị null rơi vào khoảng 2000 dòng không ảnh hưởng nhiều

In [49]:
df.dropna(inplace=True)

Kiểm tra lại dataframe

In [50]:
df.isnull().sum()

id                              0
title                           0
url                             0
is_paid                         0
num_subscribers                 0
avg_rating                      0
avg_rating_recent               0
rating                          0
num_reviews                     0
is_wishlisted                   0
num_published_lectures          0
num_published_practice_tests    0
created                         0
published_time                  0
discount_price__amount          0
discount_price__currency        0
discount_price__price_string    0
price_detail__amount            0
price_detail__currency          0
price_detail__price_string      0
dtype: int64

In [51]:
df.shape

(21024, 20)

**Loại bỏ cột không cần thiết**

Các cột bị loại bỏ:
- Cột `discount_price__currency` và `price_detail__currency` đều chỉ có 1 giá trị duy nhất là `INR`
- Cột `is_wishlisted` có 1 giá trị duy nhất là `False`
- Cột `discount_price__price_string` có dữ liệu kiểu string và được tổng hợp từ 2 cột `discount_price__currency` và `discount_price__amount` nên ta sẽ loại bỏ cột này. Tương tự với cột `price_detail__price_string`
- Cột `id`, `url` không có thông tin hữu ích cho việc trực quan hóa và mô hình hóa
- Cột `avg_rating_recent` có giá trị dữ liệu từng dòng bằng với cột `rating` nên ta chọn loại bỏ cột `avg_rating_recent` để tránh dư thừa cột 

In [52]:
print(df["discount_price__currency"].unique())
print(df["price_detail__currency"].unique())
print(df['is_wishlisted'].unique())

['INR']
['INR']
[False]


In [53]:
print(df['avg_rating_recent'].equals(df['rating']))

True


In [54]:
unused_columns = [
    'discount_price__currency', 
    'price_detail__currency',
    'discount_price__price_string',
    'price_detail__price_string',
    'is_wishlisted',
    'url',
    'id',
    'avg_rating_recent']

df.drop(columns=unused_columns, inplace=True)
df.columns

Index(['title', 'is_paid', 'num_subscribers', 'avg_rating', 'rating',
       'num_reviews', 'num_published_lectures', 'num_published_practice_tests',
       'created', 'published_time', 'discount_price__amount',
       'price_detail__amount'],
      dtype='object')

**Chuyển về kiểu dữ liệu datetime**

Áp dung cho 2 cột `created` và `published_time`

In [55]:
df["created"] = pd.to_datetime(df["created"])
df["published_time"] = pd.to_datetime(df["published_time"])

Kiểu tra kiểu dữ liệu

In [56]:
df.dtypes

title                                        object
is_paid                                        bool
num_subscribers                               int64
avg_rating                                  float64
rating                                      float64
num_reviews                                   int64
num_published_lectures                        int64
num_published_practice_tests                  int64
created                         datetime64[ns, UTC]
published_time                  datetime64[ns, UTC]
discount_price__amount                      float64
price_detail__amount                        float64
dtype: object

Cách trích xuất thông tin từ dữ liệu datetime

In [57]:
df['published_time'].dt.hour

0         5
1        16
2        23
3         1
4        21
         ..
22847    18
22848     9
22849     7
22851    16
22852     0
Name: published_time, Length: 21024, dtype: int64

**Chuyển giá trị tiền tệ từ INR thành USD**

USD là đơn vị tiền tệ phổ biến nên chuyển từ INR sang USD để có thể hiểu rõ hơn. Tại thời điểm thực hiện, 1 INR bằng khoảng 0.012 USD

In [58]:
df['discount_price__amount'] = df['discount_price__amount'].apply(lambda x: x * 0.012)
df['price_detail__amount'] = df['price_detail__amount'].apply(lambda x: x * 0.012)

In [59]:
df.head()

,title,is_paid,num_subscribers,avg_rating,rating,num_reviews,num_published_lectures,num_published_practice_tests,created,published_time,discount_price__amount,price_detail__amount
0,The Complete SQL Bootcamp 2020: Go from Zero t...,True,295509,4.66019,4.67874,78006,84,0,2016-02-14 22:57:48+00:00,2016-04-06 05:16:11+00:00,5.46,103.68
1,Tableau 2020 A-Z: Hands-On Tableau Training fo...,True,209070,4.58956,4.60015,54581,78,0,2016-08-22 12:10:18+00:00,2016-08-23 16:59:49+00:00,5.46,103.68
2,PMP Exam Prep Seminar - PMBOK Guide 6,True,155282,4.59491,4.59326,52653,292,2,2017-09-26 16:32:48+00:00,2017-11-14 23:58:14+00:00,5.46,103.68
3,The Complete Financial Analyst Course 2020,True,245860,4.54407,4.53772,46447,338,0,2015-10-23 13:34:35+00:00,2016-01-21 01:38:48+00:00,5.46,103.68
4,An Entire MBA in 1 Course:Award Winning Busine...,True,374836,4.47080,4.47173,41630,83,0,2015-10-12 06:39:46+00:00,2016-01-11 21:39:33+00:00,5.46,103.68
